# Etape 1 - Exploration et chargement Spark

**Objectif** : Charger et explorer les donnees de qualite de l'air avec PySpark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
import os

# Chemins des donnees
DATA_DIR = "../data"
AIR_QUALITY_PATH = os.path.join(DATA_DIR, "air_quality_raw.csv")
STATIONS_PATH = os.path.join(DATA_DIR, "stations.csv")
WEATHER_PATH = os.path.join(DATA_DIR, "weather_raw.csv")

## 1.1 Creation de la session Spark

In [2]:
# Creer une session Spark locale
spark = SparkSession.builder \
    .appName("TP Qualite Air - Exploration") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

# Reduire les logs
spark.sparkContext.setLogLevel("WARN")

print(f"Spark version: {spark.version}")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")

Spark version: 3.5.7
Spark UI: http://host.docker.internal:4041


## 1.2 Chargement des donnees de qualite de l'air

In [3]:
# Charger le CSV avec inference de schema
df_air_raw = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(AIR_QUALITY_PATH)

print(f"Nombre de lignes: {df_air_raw.count():,}")
print(f"Nombre de colonnes: {len(df_air_raw.columns)}")

Nombre de lignes: 1,230,951
Nombre de colonnes: 5


In [4]:
# Afficher le schema infere
print("Schema infere:")
df_air_raw.printSchema()

Schema infere:
root
 |-- station_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- pollutant: string (nullable = true)
 |-- value: string (nullable = true)
 |-- unit: string (nullable = true)



In [5]:
# Apercu des donnees
df_air_raw.show(10, truncate=False)

+----------+-------------------+---------+-----+-----+
|station_id|timestamp          |pollutant|value|unit |
+----------+-------------------+---------+-----+-----+
|ST0040    |2024-01-07T05:00:00|O3       |79.29|ug/m3|
|ST0004    |06/09/2024 18:00:00|O3       |41.58|ug/m3|
|ST0027    |2024-05-23 11:00:00|PM10     |29.20|ug/m3|
|ST0002    |18/03/2024 12:00   |SO2      |7.72 |ug/m3|
|ST0035    |2024-06-11T08:00:00|O3       |29.87|ug/m3|
|ST0023    |19/04/2024 10:00   |O3       |30.07|ug/m3|
|ST0035    |19/03/2024 05:00   |PM2.5    |14.12|ug/m3|
|ST0001    |2024-05-19T13:00:00|PM10     |17,09|ug/m3|
|ST0014    |10/03/2024 20:00   |CO       |0.29 |mg/m3|
|ST0004    |2024-01-28T16:00:00|NO2      |25,69|ug/m3|
+----------+-------------------+---------+-----+-----+
only showing top 10 rows



## 1.3 Identification des problemes de typage

In [6]:
# La colonne 'value' est en string car elle contient des virgules et des valeurs textuelles
# Examinons les valeurs non numeriques

# Valeurs qui ne peuvent pas etre converties en nombre
df_non_numeric = df_air_raw.filter(
    ~F.col("value").rlike("^-?[0-9]+[.,]?[0-9]*$")
)

print(f"Nombre de valeurs non numeriques: {df_non_numeric.count():,}")
df_non_numeric.select("value").distinct().show()

Nombre de valeurs non numeriques: 6,076
+-----+
|value|
+-----+
| null|
|  N/A|
|error|
|  ---|
+-----+



In [7]:
# Valeurs avec virgule comme separateur decimal
df_with_comma = df_air_raw.filter(F.col("value").contains(","))
print(f"Nombre de valeurs avec virgule: {df_with_comma.count():,}")
df_with_comma.select("value").show(5)

Nombre de valeurs avec virgule: 184,556
+-----+
|value|
+-----+
|17,09|
|25,69|
|15,42|
|29,14|
|12,82|
+-----+
only showing top 5 rows



In [8]:
# Differents formats de timestamp
print("Exemples de formats de timestamp:")
df_air_raw.select("timestamp").distinct().show(20, truncate=False)

Exemples de formats de timestamp:
+-------------------+
|timestamp          |
+-------------------+
|2024-01-07T05:00:00|
|2024-05-19T13:00:00|
|05/26/2024 14:00:00|
|2024-04-12T09:00:00|
|2024-01-31T14:00:00|
|2024-06-02 20:00:00|
|05/18/2024 07:00:00|
|2024-04-26T22:00:00|
|2024-03-31T02:00:00|
|28/04/2024 22:00   |
|2024-03-22 10:00:00|
|2024-04-02 00:00:00|
|2024-03-30 18:00:00|
|16/03/2024 02:00   |
|04/22/2024 22:00:00|
|03/03/2024 02:00:00|
|2024-06-28 11:00:00|
|12/05/2024 12:00   |
|17/01/2024 03:00   |
|2024-03-28T08:00:00|
+-------------------+
only showing top 20 rows



## 1.4 Statistiques descriptives par polluant

In [9]:
# Convertir value en double (en remplacant la virgule par un point)
df_air_numeric = df_air_raw.withColumn(
    "value_clean",
    F.regexp_replace(F.col("value"), ",", ".").cast("double")
)

# Statistiques par polluant (en ignorant les valeurs nulles)
stats_by_pollutant = df_air_numeric.filter(F.col("value_clean").isNotNull()) \
    .groupBy("pollutant") \
    .agg(
        F.count("*").alias("count"),
        F.round(F.mean("value_clean"), 2).alias("mean"),
        F.round(F.stddev("value_clean"), 2).alias("stddev"),
        F.round(F.min("value_clean"), 2).alias("min"),
        F.round(F.max("value_clean"), 2).alias("max"),
        F.round(F.expr("percentile(value_clean, 0.5)"), 2).alias("median")
    ) \
    .orderBy("pollutant")

print("Statistiques par polluant:")
stats_by_pollutant.show()

Statistiques par polluant:
+---------+------+------+------+-------+-------+------+
|pollutant| count|  mean|stddev|    min|    max|median|
+---------+------+------+------+-------+-------+------+
|       CO|204062| 33.71|341.16|  -2.57|4996.48|  0.71|
|      NO2|204093| 78.09|336.51|-163.48|4998.69| 42.43|
|       O3|204220|108.78|337.01|-269.95|4999.37| 70.52|
|     PM10|204133|  70.1|335.05|-132.57|4999.15| 35.25|
|    PM2.5|204189| 55.31|336.76| -81.23|4999.69| 21.18|
|      SO2|204178| 40.73|342.75| -26.57|4997.95|  7.05|
+---------+------+------+------+-------+-------+------+



In [10]:
# Identifier les valeurs aberrantes
print("Valeurs negatives:")
df_air_numeric.filter(F.col("value_clean") < 0).groupBy("pollutant").count().show()

print("\nValeurs > 1000 ug/m3:")
df_air_numeric.filter(F.col("value_clean") > 1000).groupBy("pollutant").count().show()

Valeurs negatives:
+---------+-----+
|pollutant|count|
+---------+-----+
|      SO2| 2018|
|       O3| 2062|
|      NO2| 2033|
|     PM10| 2040|
|       CO| 2087|
|    PM2.5| 2070|
+---------+-----+


Valeurs > 1000 ug/m3:
+---------+-----+
|pollutant|count|
+---------+-----+
|      SO2| 2066|
|       O3| 2080|
|      NO2| 2031|
|     PM10| 2017|
|       CO| 2070|
|    PM2.5| 2063|
+---------+-----+



## 1.5 Comptage des valeurs nulles par colonne

In [18]:
# Compter les nulls pour chaque colonne
null_counts = df_air_raw.select([
    F.count(F.when(F.col(c).isNull() | (F.trim(F.col(c)) == "") | (F.trim(F.lower(F.col(c))).isin("null", "none")), c)).alias(c)
    for c in df_air_raw.columns
])

print("Nombre de valeurs nulles/vides par colonne:")
null_counts.show()

Nombre de valeurs nulles/vides par colonne:
+----------+---------+---------+-----+----+
|station_id|timestamp|pollutant|value|unit|
+----------+---------+---------+-----+----+
|         0|        0|        0| 1538|   0|
+----------+---------+---------+-----+----+



In [19]:
# Pourcentage de completude
total_rows = df_air_raw.count()
print(f"\nTaux de completude (sur {total_rows:,} lignes):")

for col_name in df_air_raw.columns:
    null_count = df_air_raw.filter(
        F.col(col_name).isNull() | (F.trim(F.col(col_name)) == "") | (F.trim(F.lower(F.col(col_name))).isin("null", "none"))
    ).count()
    completude = (1 - null_count / total_rows) * 100
    print(f"  {col_name}: {completude:.2f}%")


Taux de completude (sur 1,230,951 lignes):
  station_id: 100.00%
  timestamp: 100.00%
  pollutant: 100.00%
  value: 99.88%
  unit: 100.00%


## 1.6 Stations avec le plus d'enregistrements

In [20]:
# Charger les stations
df_stations = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(STATIONS_PATH)

df_stations.show(10)

+----------+--------------------+---------+---------+--------+------------+
|station_id|        station_name|     city|      lat|     lon|station_type|
+----------+--------------------+---------+---------+--------+------------+
|    ST0001|     Paris-urbaine-1|    Paris|48.809101|2.329703|     urbaine|
|    ST0002| Paris-periurbaine-2|    Paris|48.828921|2.375847| periurbaine|
|    ST0003|Paris-industrielle-3|    Paris| 48.87427|2.391418|industrielle|
|    ST0004|      Lyon-urbaine-1|     Lyon|45.773049|4.788878|     urbaine|
|    ST0005|  Lyon-periurbaine-2|     Lyon| 45.72337|4.808966| periurbaine|
|    ST0006| Lyon-industrielle-3|     Lyon|45.774202|4.841825|industrielle|
|    ST0007| Marseille-urbaine-1|Marseille|43.288452|5.364721|     urbaine|
|    ST0008|Marseille-periurb...|Marseille|43.274319| 5.40673| periurbaine|
|    ST0009|Marseille-industr...|Marseille|43.322381|5.335766|industrielle|
|    ST0010|  Marseille-trafic-4|Marseille|43.288761|5.347587|      trafic|
+----------+

In [21]:
# Nombre d'enregistrements par station
records_by_station = df_air_raw.groupBy("station_id") \
    .count() \
    .orderBy(F.desc("count"))

# Joindre avec les infos des stations
records_with_info = records_by_station.join(
    df_stations,
    on="station_id",
    how="left"
).select(
    "station_id", "station_name", "city", "station_type", "count"
)

print("Top 10 stations avec le plus d'enregistrements:")
records_with_info.show(10)

Top 10 stations avec le plus d'enregistrements:
+----------+--------------------+----------+------------+-----+
|station_id|        station_name|      city|station_type|count|
+----------+--------------------+----------+------------+-----+
|    ST0019|  Bordeaux-urbaine-1|  Bordeaux|     urbaine|26164|
|    ST0016|Toulouse-industri...|  Toulouse|industrielle|26172|
|    ST0003|Paris-industrielle-3|     Paris|industrielle|26239|
|    ST0007| Marseille-urbaine-1| Marseille|     urbaine|26219|
|    ST0034| Strasbourg-trafic-4|Strasbourg|      trafic|26190|
|    ST0017|   Toulouse-trafic-4|  Toulouse|      trafic|26199|
|    ST0002| Paris-periurbaine-2|     Paris| periurbaine|26176|
|    ST0035|  Grenoble-urbaine-1|  Grenoble|     urbaine|26221|
|    ST0028|Nantes-periurbaine-2|    Nantes| periurbaine|26241|
|    ST0037|Grenoble-industri...|  Grenoble|industrielle|26233|
+----------+--------------------+----------+------------+-----+
only showing top 10 rows



In [22]:
# Repartition par ville
records_by_city = df_air_raw.join(
    df_stations.select("station_id", "city"),
    on="station_id",
    how="left"
).groupBy("city") \
 .count() \
 .orderBy(F.desc("count"))

print("Repartition des enregistrements par ville:")
records_by_city.show()

Repartition des enregistrements par ville:
+----------+------+
|      city| count|
+----------+------+
|     Rouen|183348|
| Marseille|183294|
|  Grenoble|157073|
|  Toulouse|130979|
|     Lille|130897|
|    Nantes|104840|
|Strasbourg|104791|
|     Paris| 78611|
|  Bordeaux| 78574|
|      Lyon| 78544|
+----------+------+



## 1.7 Synthese des problemes de qualite identifies

In [23]:
# Resume des problemes
total = df_air_raw.count()

# Valeurs non numeriques
non_numeric = df_air_raw.filter(
    ~F.col("value").rlike("^-?[0-9]+[.,]?[0-9]*$")
).count()

# Valeurs avec virgule
with_comma = df_air_raw.filter(F.col("value").contains(",")).count()

# Valeurs negatives (apres conversion)
negative = df_air_numeric.filter(F.col("value_clean") < 0).count()

# Valeurs aberrantes > 1000
outliers = df_air_numeric.filter(F.col("value_clean") > 1000).count()

# Doublons
duplicates = total - df_air_raw.dropDuplicates(["station_id", "timestamp", "pollutant"]).count()


print(f"Total enregistrements: {total:,}")
print()
print(f"Problemes identifies:")
print(f"  - Valeurs non numeriques: {non_numeric:,} ({non_numeric/total*100:.2f}%)")
print(f"  - Valeurs avec virgule decimale: {with_comma:,} ({with_comma/total*100:.2f}%)")
print(f"  - Valeurs negatives: {negative:,} ({negative/total*100:.2f}%)")
print(f"  - Valeurs aberrantes (>1000): {outliers:,} ({outliers/total*100:.2f}%)")
print(f"  - Doublons: {duplicates:,} ({duplicates/total*100:.2f}%)")
print(f"  - Formats de dates multiples: 4 formats differents detectes")

Total enregistrements: 1,230,951

Problemes identifies:
  - Valeurs non numeriques: 6,076 (0.49%)
  - Valeurs avec virgule decimale: 184,556 (14.99%)
  - Valeurs negatives: 12,310 (1.00%)
  - Valeurs aberrantes (>1000): 12,327 (1.00%)
  - Doublons: 24,136 (1.96%)
  - Formats de dates multiples: 4 formats differents detectes


In [24]:
# Fermer la session Spark
spark.stop()